# References

In this notebook we will show several types of references that can be used as the start of a processing chain. Remember from the [Intro notebook](intro.ipynb) that result instructions are formulated by combining basic building blocks into processing chains. These processing chains start with a *reference*. At the query recipe construction stage, such a reference is nothing more than a small piece of text. When executing the recipe, the query processor solves this reference and evaluates it internally into a multi-dimensional array filled with data values, i.e. into a *data cube*. Several actions can then be applied to this data cube. For a description of those, see the [Verbs notebook](verbs.ipynb).

## Content

- [Referencing single data cubes](#Referencing-single-data-cubes)
    - [Referencing semantic concepts](#Referencing-semantic-concepts)
    - [Referencing factbase resources](#Referencing-factbase-resources)
    - [Referencing results](#Referencing-results)
    - [Referencing the active evaluation object](#Referencing-the-active-evaluation-object)
- [Referencing collections of data cubes](#Referencing-collections-of-data-cubes)

## Prepare

Import the semantique package:

In [1]:
import semantique as sq

Import other packages we will use in this notebook:

In [2]:
import geopandas as gpd
import json

Create the components for query processing (except the query recipe itself). See the [Intro notebook](intro.ipynb) for details.

In [3]:
# Ontology.
with open("files/ontology.json", "r") as file:
    ontology = sq.ontology.Semantique(json.load(file))

# Factbase.
with open("files/factbase.json", "r") as file:
    factbase = sq.factbase.GeotiffArchive(json.load(file), src = "files/resources.zip")

# Extent.
space = sq.SpatialExtent(gpd.read_file("files/footprint.geojson"))
time = sq.TemporalExtent("2019-01-01", "2020-12-31")

# Additional configuration.
config = {"crs": 3035, "tz": "UTC", "spatial_resolution": [-10, 10]}

## Referencing single data cubes

Most references are a reference that will be evaluated into a *single* data cube during query processing. At the stage of query construction, all these references are just textual references in a dictionary-like structure. More specifically, they are objects of class `CubeProxy`, meaning that it will be evaluated into a data cube, but only when executing the query recipe. There are several types of single-cube references, which we will discuss one-by-one below.

### Referencing semantic concepts

The most common reference at the start of a processing chain is a reference to a semantic concept that is defined in the ontology. You can refer to any semantic concept by using the `sq.concept` function. How to specify the reference, depends on the structure of the ontology. Usually, an ontology does not only list rulesets of semantic concepts, but also formalizes a categorization of these concepts. That is, a reference to a specific semantic concept usually consists of the name of that concept, *and* the name of the category it belongs to. Optionally there can be multiple hierarchies of categories, for example to group concepts of different semantic levels (e.g. an entity *water body* is of a lower semantic level than an entity *lake*, since lake is by definition always a water body, but a water body not necessarily a lake).  The `sq.concept` function lets you specify as many levels as you need, starting with the lowest-level category, and ending with the name of the semantic concept itself.

The common lowest-level categorization groups the semantic concepts into very abstract types. For example, a semantic concept might be an *entity* (a phenonemon with a distinct and independent *existence*, e.g. a forest or a lake) or an *event* (a phenonemon that *takes place*, e.g. a fire or a flood). If the semantic concepts are stored as direct element of these lowest-level categories without any further subdivision (as is the case in our example ontology we use here), we can refer to a semantic concept such as *water* as follows.

In [4]:
ref = sq.concept("entity", "water")
print(json.dumps(ref, indent = 2))

{
  "type": "concept",
  "reference": [
    "entity",
    "water"
  ]
}


In [5]:
type(ref)

semantique.blocks.CubeProxy

If you use ontologies that include sub-categories, you can simply use the same function to refer to them, in a form as below. There is no limit on how many sub-categories you can use in a reference. Of course, this all depends on the categorization of the ontology that you will use.

In [6]:
ref = sq.concept("entity", "natural_entities", "water_bodies", "lake")
print(json.dumps(ref, indent = 2))

{
  "type": "concept",
  "reference": [
    "entity",
    "natural_entities",
    "water_bodies",
    "lake"
  ]
}


For convenience, commonly used lowest-level semantic concept categories are also implemented as separate construction functions, such that you can call them directly. Currently, you can refer directly to the following common categories:

- `entity`: A phenonemon with a distinct and independent *existence*, e.g. a forest or a lake. We also use this category for broader, lower-level concepts such as vegetated areas (i.e. *vegetation*) and water bodies (i.e. *water*).
- `event`: A phenonemon that *takes place*, e.g. a fire or a flood.

Hence, the code below produces the same output as above.

In [7]:
ref = sq.entity("water")
print(json.dumps(ref, indent = 2))

{
  "type": "concept",
  "reference": [
    "entity",
    "water"
  ]
}


The query processor will solve these kind of references by looking up the textual reference in the ontology, obtaining the corresponding ruleset, and evualate these rules into a set of data values. If the rules describe *binary relationships* between the semantic concepts and the pixel values, the query processor will translate a semantic concept reference into a *boolean* data cube, with "true" values (i.e. 1) for those pixels that are identified as being an observation of the referenced concept, and "false" values (i.e. 0) for all other pixels in the spatio-temporal extent.

To show this, we execute a query recipe that only contains a single semantic concept reference.

In [8]:
sq.QueryRecipe({"water": ref}).execute(factbase, ontology, space, time, **config)["water"]

<xarray.DataArray 'water' (time: 3, y: 563, x: 576)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * time          (time) datetime64[ns] 2019-12-15T10:17:33.408715 ... 2020-1...
    spatial_ref   int64 0
    temporal_ref  int64 0
    feature       (y, x) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
  * y             (y) float64 2.697e+06 2.697e+06 ... 2.691e+06 2.691e+06
  * x             (x) float64 4.53e+06 4.53e+06 4.53e+06 ... 4.536e+06 4.536e+06
Attributes:
    value_type:  binary

Each semantic concept is defined by one or more named *properties* it has. For example, a entity *lake* may be defined by its *color* (a blueish, water-like color) in combination with its *texture* (it has an approximately flat surface). In the case of modelling binary relationships, the rules belonging to a property then define for each pixel in the data if it meets a specific property ("true"), or not ("false"). A pixel is identified as part of a semantic concept only if *all* properties are met.

 If you are only interested in a single property of a semantic concept, you can specify that through the `property` argument of the `sq.concept` function.

In [9]:
ref = sq.entity("water", property = "color")
print(json.dumps(ref, indent = 2))

{
  "type": "concept",
  "reference": [
    "entity",
    "water"
  ],
  "property": "color"
}


Since in our ontology the "water" concept is *exclusively* defined by its "color" property, the resulting data cube will in this case be equal to the full "water" cube we obtained before.

In [10]:
cube = sq.QueryRecipe({"color": ref}).execute(factbase, ontology, space, time, **config)["color"]
cube

<xarray.DataArray 'color' (time: 3, y: 563, x: 576)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * time          (time) datetime64[ns] 2019-12-15T10:17:33.408715 ... 2020-1...
    spatial_ref   int64 0
    temporal_ref  int64 0
    feature       (y, x) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
  * y             (y) float64 2.697e+06 2.697e+06 ... 2.691e+06 2.691e+06
  * x             (x) float64 4.53e+06 4.53e+06 4.53e+06 ... 4.536e+06 4.536e+06
Attributes:
    value_type:  binary

### Referencing factbase resources

Normally, the semantic querying structure should allow you to not worry at all about the data stored in the factbase (i.e. the *image domain*). It is the role of the ontology to combine these resources into meaningful real-world concepts (i.e. a mapping from the *image domain* to the *real-world domain*). However, especially when you are an expert user, you don't want to be limited by this structure. There are still cases in which it is useful to directly refer to factbase resources in your result instructions. 

Just as with ontologies, layout files of factbases usually formalize a categorization of resources. Such categories group resources based on what property of the observed phenonemon they describe. For example, there are often resources that tell you something about the *reflectance* of radiation from the Earth surface. When data are semantically enriched, they also contain information layers that tell you something about the *appearance* of the observed phenonemen. Hence, what does it look like? 

Because of this categorization, a reference to a specific resource of data usually consists of the name of that resource, *and* the name of the category it belongs to. Optionally there can be multiple hierarchies of categories. The `sq.resource` function lets you specify as many levels as you need, starting with the lowest-level category, and ending with the name of the resource itself. For example:

In [11]:
ref = sq.resource("appearance", "Color type")
print(json.dumps(ref, indent = 2))

{
  "type": "resource",
  "reference": [
    "appearance",
    "Color type"
  ]
}


If you use factbase layout files that include sub-categories, you can simply use the same function to refer to them, in a form as below. There is no limit on how many sub-categories you can use in a reference. Of course, this all depends on the categorization of the factbase layout that you will use.

In [12]:
ref = sq.resource("appearance", "categorical_layers", "colors", "Color type")
print(json.dumps(ref, indent = 2))

{
  "type": "resource",
  "reference": [
    "appearance",
    "categorical_layers",
    "colors",
    "Color type"
  ]
}


For convenience, commonly used factbase categories are also implemented as separate construction functions, such that you can call them directly. Currently, you can refer directly to the following common categories:

- `appearance`: Resources that contain information on what the observed phenonemon looks like.
- `reflectance`: Resources that contain information on how intensely certain types of radiation are reflected by the observed phenonemon.
- `topography`: Resources that contain information on the surface form of the observed phenonemon.
- `atmosphere`: Resources that contain information on what the atmosphere above the observed phenonemon looks like.
- `artifacts`: Resources that label erroneous observations.

Hence, the code below produces the same output as above.

In [13]:
ref = sq.appearance("Color type")
print(json.dumps(ref, indent = 2))

{
  "type": "resource",
  "reference": [
    "appearance",
    "Color type"
  ]
}


The query processor will solve these kind of references by looking up the textual reference in the layout file of the factbase, obtaining the corresponding metadata object, and using the information in there to retrieve the corresponding data values from the factbase.

To show this, we execute a query recipe that only contains a single factbase resource reference.

In [14]:
cube = sq.QueryRecipe({"colortype": ref}).execute(factbase, ontology, space, time, **config)["colortype"]
cube

<xarray.DataArray 'colortype' (time: 3, y: 563, x: 576)>
array([[[29., 29., 29., ..., 29., 29., 29.],
        [29., 29., 29., ..., 29., 29., 29.],
        [29., 29., 29., ...,  5., 29., 29.],
        ...,
        [ 5.,  5.,  5., ..., 29., 29., 29.],
        [ 5.,  5.,  5., ..., 29., 29., 29.],
        [ 5., 29., 29., ..., 29., 29., 29.]],

       [[ 4.,  4.,  4., ...,  4.,  1.,  1.],
        [ 6.,  4.,  4., ...,  3.,  1.,  4.],
        [ 6.,  4.,  4., ...,  4.,  4.,  4.],
        ...,
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

       [[21., 21., 21., ...,  4.,  5.,  4.],
        [21., 21., 21., ...,  3.,  6.,  4.],
        [21., 21., 21., ...,  3.,  4.,  4.],
        ...,
        [ 3.,  3.,  3., ...,  3., 27., 27.],
        [ 3., 27.,  3., ..., 27., 27., 27.],
        [27.,  3.,  7., ..., 27., 27., 27.]]])
Coordinates:
  * time          (time) datetime64[ns] 2019-12-15T10:17:33.408715 ... 2020-1...
    spatial_ref   int64 0
    temporal_ref  int64 0
    feature       (y, x) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
  * y             (y) float64 2.697e+06 2.697e+06 ... 2.691e+06 2.691e+06
  * x             (x) float64 4.53e+06 4.53e+06 4.53e+06 ... 4.536e+06 4.536e+06
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     33SharedSpCt
    _FillValue:    1.7976931348623157e+308
    value_type:    ordinal
    value_labels:  {1: 'SVHNIR', 2: 'SVLNIR', 3: 'AVHNIR', 4: 'AVLNIR', 5: 'W...

### Referencing results

A semantic query recipe can contain multiple result instructions. Whenever a result instruction you are constructing gets too long or complicated to be defined in one line, you can of course break it up in several assignment operations. For example:

```python
water_count = sq.entity("water").reduce("time", "count")
vegetation_count = sq.entity("vegetation").reduce("time", "count")
recipe = sq.QueryRecipe({"summed_count": water_count.evaluate("add", vegetation_count)})
```

After executing this recipe, the response will contain a single result, i.e. "summed_count". However, you might also want to obtain the water count and vegetation count maps as separate results themselves. At that point, it is good to know that you can start processing chains with a reference to a previously defined result, using the `sq.result` function. This allows you to re-use other result definitions inside a new result definition. The only thing you have to use as a reference is the name of the result.

In [15]:
ref = sq.result("water_count")
print(json.dumps(ref, indent = 2))

{
  "type": "result",
  "name": "water_count"
}


The same "summed_count" result as we showed above can in this way also be formulated as shown below. The only difference here is that next to "summed_count", the response will also contain "water_count" and "vegetation_count" as separate results. When that is what you want, use the structure below. If not, use the structure above.

```python
recipe = sq.QueryRecipe()
recipe["water_count"] = sq.entity("water").reduce("time", "count")
recipe["vegetation_count"] = sq.entity("vegetation").reduce("time", "count")
recipe["summed_count"] = sq.result("water_count").evaluate("add", sq.result("vegetation_count"))
```

### Referencing the active evaluation object

Before explaining these type of references, we need to introduce some slightly more advanced processing chain structures. Remember that a processing chain starts with a reference, and that this reference will internally be evaluated into a data cube. Specific actions (i.e. the [verbs](verbs.ipynb)) can then be applied to this data cube. This can be a single action, but also a chain of multiple actions. The data cube constantly changes when moving through the processing chain. It starts as the evaluated reference. This object will be the input to the first action, which wrangles it into a different cube. That different cube will then be the input to the second action, which again wrangles it into a different cube, etcetera. We use the term **active evaluation object** to refer to the input object at each stage of the processing chain. Hence, at the first action, the active evaluation object is the evaluated reference. At the second action, the active evaluation object is the output of the first action, etcetera.

Some actions combine information of multiple inputs. For example, the [evaluate verb](https://zgis.github.io/semantique/_notebooks/verbs.html#Evaluate) lets you evaluate multivariate expressions where the left-hand side of the expression is the active evaluation object, and the right-hand side of the expression is either a single value or another data cube. For example, adding a vegetation count to a water count:

```python
sq.entity("water").reduce("time", "count").evaluate("add", sq.entity("vegetation").reduce("time", "count"))
```

The processing chain `sq.entity("vegetation").reduce("time", "count")` here is nested into the main processing chain that starts with the reference to water. Since `sq.entity("water").reduce("time", "count")` is the active evaluation object of the main processing chain at the stage of the `evaluate` verb, the final outcome of the line above is the vegetation count added to the water count, i.e. the sum of both counts.

But now what if we want to add the water count to itself? Our recipe would look like this:

```python
sq.entity("water").reduce("time", "count").evaluate("add", sq.entity("water").reduce("time", "count"))
```

Here you see that the *same* processing chain `sq.entity("water").reduce("time", "count")` occurs twice in the recipe. This does not only make the code longer and less readable, it also decreases processing time since solving the water reference and reducing the resulting cube has to be done twice instead of once. Now we came at the point where we can show that it is useful to reference the active evaluation object itself. You can use the `sq.self` function for that, without the need to provide any arguments.

In [16]:
ref = sq.self()
print(json.dumps(ref, indent = 2))

{
  "type": "self"
}


This allows us to simplify the recipe above (and improve processing speed!):

```python
sq.entity("water").reduce("time", "count").evaluate("add", sq.self())
```

Of course, the example above is trivial. Why would you add the active evaluation object to itself, when you can also just multiply it by 2. However, there are cases where the self-reference can be of real use. For example, when applying a self-filter (see [here](https://zgis.github.io/semantique/_notebooks/verbs.html#Self-filtering)), when applying a filter based on dimension coordinates (see [here](https://zgis.github.io/semantique/_notebooks/verbs.html#Aligning-the-filterer)), or when grouping a data cube along one of its dimensions (see [here](https://zgis.github.io/semantique/_notebooks/verbs.html#Groupby).

It may be clear now that references to the active evaluation object itself are only meant to be used at the start of *nested* processing chains. It is of no use to start a main processing chain with a self reference, since there is no active evaluation object yet. Although, technically there is. The query processor stores the [spatio-temporal extent cube](https://zgis.github.io/semantique/_notebooks/advanced.html#Creating-a-spatio-temporal-extent-cube) of the query as the "default" active evaluation object. That is, when you refer to the active evaluation object outside of a nested structure, you will obtain the spatio-temporal extent cube:

In [17]:
cube = sq.QueryRecipe({"self": ref}).execute(factbase, ontology, space, time, **config)["self"]
cube

<xarray.DataArray 'self' (time: 2, y: 563, x: 576)>
array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]])
Coordinates:
  * time          (time) datetime64[ns] 2019-01-01 2020-12-31
    spatial_ref   int64 0
    feature       (y, x) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    temporal_ref  int64 0
  * y             (y) float64 2.697e+06 2.697e+06 ... 2.691e+06 2.691e+06
  * x             (x) float64 4.53e+06 4.53e+06 4.53e+06 ... 4.536e+06 4.536e+06
Attributes:
    name:          index
    long_name:     index
    _FillValue:    nan
    value_type:    nominal
    value_labels:  {1: 'feature_1'}

## Referencing collections of data cubes

Up to now we have only talked about references that are evaluated into a *single* data cube. It is also possible to reference a *collection* of multiple data cubes at once. Such data cube collections have [dedicated verbs](https://zgis.github.io/semantique/_notebooks/verbs.html#Verbs-for-data-cube-collections) that can be applied to them, which will all in some way combine all collection members into a single data cube.

Consider for example a case where you want to sum five data cubes together. You could do this by starting a processing chain with the first of those cubes, and calling four times the [evaluate verb](https://zgis.github.io/semantique/_notebooks/verbs.html#Evaluate) to add each of the other cubes step-by-step. It is much easier to reference all the five cubes together in a collection, and call the [merge verb](https://zgis.github.io/semantique/_notebooks/verbs.html#Merge) to add them together in a single step. Another reason to use collections is related to style. A chained structure always implies some kind of a hierarchy, in which you start with a main input object, and add the other objects along the way. This might not feel fitting for every use-case. 

To reference a collection of data cubes, you can use the `sq.collection` function. As arguments you give it as many single-cube references as you want. For example:

In [18]:
ref = sq.collection(sq.entity("water"), sq.entity("vegetation"))
print(json.dumps(ref, indent = 2))

{
  "type": "collection",
  "elements": [
    {
      "type": "concept",
      "reference": [
        "entity",
        "water"
      ]
    },
    {
      "type": "concept",
      "reference": [
        "entity",
        "vegetation"
      ]
    }
  ]
}


Where all single-cube references return an object of class `CubeProxy`, a collection reference returns an object of class `CubeCollectionProxy`. This allows you to call the collection-specific verbs on the reference.

In [19]:
type(ref)

semantique.blocks.CubeCollectionProxy

Normally a query recipe that involves collections will combine collections members into a single cube, using the dedicated [collection verbs](https://zgis.github.io/semantique/_notebooks/verbs.html#Verbs-for-data-cube-collections). However, it is technically also possible to return a data cube collection as a result.

In [20]:
cubes = sq.QueryRecipe({"collection": ref}).execute(factbase, ontology, space, time, **config)["collection"]

In [21]:
cubes[0]

<xarray.DataArray 'water' (time: 3, y: 563, x: 576)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * time          (time) datetime64[ns] 2019-12-15T10:17:33.408715 ... 2020-1...
    spatial_ref   int64 0
    temporal_ref  int64 0
    feature       (y, x) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
  * y             (y) float64 2.697e+06 2.697e+06 ... 2.691e+06 2.691e+06
  * x             (x) float64 4.53e+06 4.53e+06 4.53e+06 ... 4.536e+06 4.536e+06
Attributes:
    value_type:  binary

In [22]:
cubes[1]

<xarray.DataArray 'vegetation' (time: 3, y: 563, x: 576)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]])
Coordinates:
  * time          (time) datetime64[ns] 2019-12-15T10:17:33.408715 ... 2020-1...
    spatial_ref   int64 0
    temporal_ref  int64 0
    feature       (y, x) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
  * y             (y) float64 2.697e+06 2.697e+06 ... 2.691e+06 2.691e+06
  * x             (x) float64 4.53e+06 4.53e+06 4.53e+06 ... 4.536e+06 4.536e+06
Attributes:
    value_type:  binary